In [2]:
import pandas as pd

# 1. Charger les fichiers
df_stats = pd.read_csv("../data/raw/fbref_final_basic_stats.csv", sep=';')
df_titles = pd.read_csv("../data/raw/titles_2025.csv")
  # le fichier des titres

# 2. Fusionner les deux sur le nom du joueur
df = df_stats.merge(df_titles, on="Player", how="left")

# 3. Remplir les valeurs NaN par 0 pour les colonnes de titres
df[["DomesticTitles", "EuropeanTitles", "NationalTeamWin", "TeamTitles"]] = df[
    ["DomesticTitles", "EuropeanTitles", "NationalTeamWin", "TeamTitles"]
].fillna(0)

# 4. Aperçu des données fusionnées
print("✅ Fusion réussie. Exemple :")
df[["Player", "Gls", "TeamTitles"]].head()


✅ Fusion réussie. Exemple :


,Player,Gls,TeamTitles
0,Federico Valverde,6.0,2.0
1,Kylian Mbappé,29.0,2.0
2,Thibaut Courtois,0.0,0.0
3,Aurélien Tchouaméni,0.0,2.0
4,Jude Bellingham,9.0,2.0


In [4]:
print(df.columns.tolist())


['Player', 'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1', 'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1', 'npxG+xAG.1', 'Unnamed: 33', 'MP.1', 'DomesticTitles', 'EuropeanTitles', 'NationalTeamWin', 'TeamTitles', 'GoalsPer90', 'xGPer90']


In [5]:
# Éviter la division par zéro
df["Min"] = df["Min"].replace(0, pd.NA)

# Calculs des indicateurs par 90 minutes
df["GoalsPer90"] = df["Gls"] / (df["Min"] / 90)
df["xGPer90"] = df["xG"] / (df["Min"] / 90)
df["xAPer90"] = df["xAG"] / (df["Min"] / 90)
df["xGContribution"] = (df["xG"] + df["xAG"]) / (df["Min"] / 90)

# Remplacer les valeurs NaN (joueurs sans minutes) par 0
df.fillna(0, inplace=True)

print("✅ Indicateurs GoalsPer90, xGPer90 et xGContribution ajoutés.")


✅ Indicateurs GoalsPer90, xGPer90 et xGContribution ajoutés.


In [11]:
# Création du FairPlayScore basé sur les cartons
# Hypothèse : 1 rouge ≈ 3 jaunes

df["FairPlayScore"] = 10 - (df["CrdY"].fillna(0) + 3 * df["CrdR"].fillna(0))

# Empêcher les scores négatifs (max entre 0 et la valeur calculée)
df["FairPlayScore"] = df["FairPlayScore"].clip(lower=0)

# Aperçu
df[["Player", "CrdY", "CrdR", "FairPlayScore"]].head()


,Player,CrdY,CrdR,FairPlayScore
0,Federico Valverde,4.0,0.0,6.0
1,Kylian Mbappé,3.0,1.0,4.0
2,Thibaut Courtois,1.0,0.0,9.0
3,Aurélien Tchouaméni,5.0,0.0,5.0
4,Jude Bellingham,5.0,1.0,2.0


In [14]:
# Extraire uniquement la partie avant le tiret, gérer les NaN, puis convertir proprement
df["Age"] = (
    df["Age"]
    .astype(str)
    .str.extract(r"^(\d+)-")      # capture les chiffres avant le "-"
    .astype(float)                # permet les NaN
    .round(0).astype("Int64")     # type nullable pour éviter les erreurs
)



In [16]:
df.to_csv("../data/raw/ballondor_dataset.csv", index=False)
print("✅ Fichier enrichi final sauvegardé.")


✅ Fichier enrichi final sauvegardé.
